# Unsupervised learning

### The goal of the notebook is to research some models x features sets out of *.pcaps

In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import collections
import seaborn as sns
import pprint
import os
import ipywidgets
import warnings

import pyshark
import networkx as nx

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances, silhouette_score
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
import umap

from itertools import product

# PATH change to access library cyberlib
import sys
sys.path.append('/home/benjamin/Folders_Python/Cyber/libs')
import cyberlib as cbl

# to allow PyShark to run in Jupyter notebooks
# import nest_asyncio
# nest_asyncio.apply()

/home/benjamin/anaconda3/envs/cyber/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/benjamin/anaconda3/envs/cyber/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/benjamin/anaconda3/envs/cyber/lib/python3.10/site-packages/uma

In [31]:
cli="""tshark \
    -r ~/Folders_Python/Cyber/data/input_pcaps/test.pcap \
    -2 \
    -R "tcp" \
    -T fields -E header=n -E separator=, \
    -e frame.number -e eth.src -e eth.dst \
    -e ip.src_host -e ip.dst_host -e tcp.flags \
    -e ip -e ip.len -e ip.hdr_len -e ip.ttl -e ip.proto \
    -e tcp.srcport -e tcp.dstport -e tcp.stream -e tcp.len \
    -e tcp.seq -e tcp.ack -e tcp.hdr_len -e tcp.time_relative \
    -e tcp.time_delta \
    -o 'gui.column.format:"No","%m","Time","%t","Source","%s","Destination","%d","Protocol","%p","Length","%L","Info","%i"' \
    > ~/Folders_Python/Cyber/data/input_pcaps/test.csv"""

os.system(cli)

0

In [12]:
# logging set-up

lg = cbl.GetLogger('/home/benjamin/Folders_Python/Cyber/logs/unsupervised_learning.log')
logger = lg.get_custom_logger()

# start your engine
logger.info("-------- new run --------")

### Get the *.pcap to play with

In [13]:
DFNAME = 'exemple'

PCAPFILE = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/' + DFNAME + '.pcap'

In [ ]:
# capture = pyshark.FileCapture(
#     input_file=PCAPFILE,
#     use_ek=False
# )

# logger.info(f'-- created a capture object in PyShark with pcap file = {PCAPFILE} --')

In [ ]:
# pp = pprint.PrettyPrinter(indent=4)

# for id in range(5):
#     p = cbl.PyPacket(capture[id])
#     pp.pprint(p.data)
#     print('-------------')

### Choice of features

We will look at Ethernet packets part of a TCP conversation

Features at TCP level : source port, destination port, sequence number, acknowledgement number, flags, header length, total length, time delta since last packet, time delta since first packet

Features at IP level : source IP, destination IP, flags, header length, length, identification, ttl, version

Features at ETH level : None

In [32]:
filename = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/test.csv'

with open(file=filename, encoding='utf-8') as f:
    df_raw = pd.read_csv(f, sep=",")

In [33]:
df_raw

,1,f8:1e:df:e5:84:3a,00:1f:f3:3c:e1:13,172.16.11.12,74.125.19.17,0x0018,Internet Protocol Version 4,Src: 172.16.11.12,Dst: 74.125.19.17,79,...,6,64565,443,0,27,1.1,1.2,32,0.000000000,0.000000000.1
0,2,f8:1e:df:e5:84:3a,00:1f:f3:3c:e1:13,172.16.11.12,74.125.19.17,0x0011,Internet Protocol Version 4,Src: 172.16.11.12,Dst: 74.125.19.17,52,...,6,64565,443,0,0,28,1,32,0.000722,0.000722
1,3,00:1f:f3:3c:e1:13,f8:1e:df:e5:84:3a,74.125.19.17,172.16.11.12,0x0010,Internet Protocol Version 4,Src: 74.125.19.17,Dst: 172.16.11.12,52,...,6,443,64565,0,0,1,28,32,0.021577,0.020855
2,4,f8:1e:df:e5:84:3a,00:1f:f3:3c:e1:13,172.16.11.12,74.125.19.17,0x0011,Internet Protocol Version 4,Src: 172.16.11.12,Dst: 74.125.19.17,52,...,6,64565,443,0,0,28,1,32,0.021626,0.000049
3,5,00:1f:f3:3c:e1:13,f8:1e:df:e5:84:3a,74.125.19.17,172.16.11.12,0x0011,Internet Protocol Version 4,Src: 74.125.19.17,Dst: 172.16.11.12,52,...,6,443,64565,0,0,1,28,32,0.022584,0.000958
4,6,f8:1e:df:e5:84:3a,00:1f:f3:3c:e1:13,172.16.11.12,74.125.19.17,0x0011,Internet Protocol Version 4,Src: 172.16.11.12,Dst: 74.125.19.17,52,...,6,64565,443,0,0,28,2,32,0.022611,0.000027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,102,00:1f:f3:3c:e1:13,f8:1e:df:e5:84:3a,96.17.211.172,172.16.11.12,0x0010,Internet Protocol Version 4,Src: 96.17.211.172,Dst: 172.16.11.12,52,...,6,80,64583,3,0,293,1053,32,0.535854,0.023326
101,103,00:1f:f3:3c:e1:13,f8:1e:df:e5:84:3a,96.17.211.172,172.16.11.12,0x0018,Internet Protocol Version 4,Src: 96.17.211.172,Dst: 172.16.11.12,322,...,6,80,64585,5,270,270,1037,32,0.347098,0.027852
102,104,f8:1e:df:e5:84:3a,00:1f:f3:3c:e1:13,172.16.11.12,96.17.211.172,0x0010,Internet Protocol Version 4,Src: 172.16.11.12,Dst: 96.17.211.172,52,...,6,64585,80,5,0,1037,540,32,0.347122,0.000024
103,105,00:1f:f3:3c:e1:13,f8:1e:df:e5:84:3a,96.17.211.172,172.16.11.12,0x0018,Internet Protocol Version 4,Src: 96.17.211.172,Dst: 172.16.11.12,322,...,6,80,64583,3,270,293,1053,32,0.537892,0.002038


In [ ]:
DIRDATAFRAMES = '/home/benjamin/Folders_Python/Cyber/data/dataframes/'
filename = DIRDATAFRAMES + 'df_raw_' + DFNAME + '.pkl'

if os.path.isfile(filename):
    df_raw = pd.read_pickle(filename)
    logger.info('read an existing dataframe for unsupervised learning')
else:
    df_raw = cbl.GetTCPDataframeFromFileCapture(filecapture=capture).dataframe
    df_raw.to_pickle(filename)
    logger.info('created and wrote a dataframe for unsupervised learning')

In [ ]:
df_raw

### Preprocessing : ordinal encoding, normalization

In [ ]:
# ordinal encoding with Pandas

columns_to_encode_as_ordinal = ['ETH_dst', 'ETH_src', 'IP_id', 'IP_flags', 'IP_src', 'IP_dst']

df_ord = pd.DataFrame()
for c in columns_to_encode_as_ordinal:
    codes, _ = pd.factorize(df_raw[c])
    df_sup = pd.DataFrame(data={ c : list(codes) })
    df_ord = pd.concat([df_ord, df_sup], axis=1)
    
df = df_raw.drop(columns=columns_to_encode_as_ordinal)
df.reset_index(drop=True)

df = pd.concat([df, df_ord], axis=1)

columns_to_drop = ['TIMESTAMP_ts']
df.drop(columns=columns_to_drop, inplace=True)

df

In [ ]:
# the rest of the preprocessing with a scikit learn pipeline

pipe = Pipeline(
    [ ('normalize', StandardScaler() )]
)

In [ ]:
X = df.to_numpy()
X_norm = pipe.fit_transform(X)

In [ ]:
X_norm

In [ ]:
def viz3d(X_embedding, titre='visu 3D'):
    """Utility function to plot a 3D picture

    Args:
        X_embedding (_type_): _description_
    """
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes(projection='3d')
    
    xs = X_embedding[:,0]
    ys = X_embedding[:,1]
    zs = X_embedding[:,2]
    
    ax.scatter3D(xs, ys, zs)
    ax.set_title(titre)
    
    plt.show()
    
    return fig, ax

### Vizualization : PCA, t-SNE

In [ ]:
pca = PCA(
    n_components = 3
)

X_new = pca.fit_transform(X_norm)

expl = [ r*100 for r in pca.explained_variance_ratio_ ]
expl_s = [ f'{e:.1f}%' for e in expl ]
print(f'Variance per principal component : {expl_s}%')

viz3d(X_new, 'PCA')
plt.show()

In [ ]:
X_new = TSNE(
    n_components=3, 
    learning_rate='auto',
    perplexity=10  # reltaed to the number of neighbors used in other manifold learning algorithms. Highly sensitve parameter.
    ).fit_transform(X_norm)

viz3d(X_new, 't-SNE')
plt.show()

### Clustering

k-means

In [ ]:
# finding a good number of clusters

MAX_CLUSTERS = 100
range_clusters = range(2, MAX_CLUSTERS+1)

silhouette = np.zeros(shape=(MAX_CLUSTERS-1))

for n_clusters in range_clusters:
    kmeans = KMeans(
        n_clusters=n_clusters,
        random_state=0,
        init='k-means++', # educated attempt to have a good initialization
        n_init=1  # because k-means++
        )
    labels = kmeans.fit_predict(X_norm)
    silhouette[n_clusters-2] = silhouette_score(X_norm, labels)
    print(f'Silhouette score for {n_clusters} clusters = {silhouette[n_clusters-2]}')
    
n_cluster_opt = np.argmax(silhouette) + 2
print(f'Nb clusters retenu = {n_cluster_opt}')

In [ ]:
fig, ax = plt.subplots()

ax.plot(silhouette)
ax.grid(True)
ax.set_title('Silhouette score v number of kmeans clusters')
plt.show()

In [ ]:
kmeans = KMeans(
    n_clusters=n_cluster_opt,
    random_state=0,
    init='k-means++', # educated attempt to have a good initialization
    n_init=1  # because k-means++
)

In [ ]:
def visu3d(X_norm=X_norm, model=None, labels=None, titre='Visu 3d'):
    """Utility function to plot the 3d PCA projection of the trained model with clusters labels

    Args:
        X_norm (_type_, optional): _description_. Defaults to X_norm.
        model (_type_, optional): _description_. Defaults to None.
        labels (_type_, optional): _description_. Defaults to None.
    """
    
    unique_labels = np.unique(labels)
    
    pca = PCA(n_components = 3)
    X_embedding = pca.fit_transform(X_norm)
    
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes(projection='3d')

    xs = X_embedding[:,0]
    ys = X_embedding[:,1]
    zs = X_embedding[:,2]

    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
    c = [ colors[labels[i]] if labels[i] >= 0 else [0,0,0,1] for i in range(len(labels)) ]
        
    ax.scatter3D(xs, ys, zs, color=c)
    n_clusters = len(set(unique_labels)) - (1 if -1 in labels else 0)
    titre = titre + f' {n_clusters} clusters'
    ax.set_title(titre)
        
    plt.show()
    
    return fig, ax

In [ ]:
labels = kmeans.fit_predict(X_norm)

fig, ax = visu3d(
    X_norm=X_norm,
    model=kmeans,
    labels=kmeans.fit_predict(X_norm),
    titre=f'kmeans'
)

DBSCAN

In [ ]:
# finding a good pair of Epsilon x Min_samples parameters

# epsilons
distances = pairwise_distances(X_norm, X_norm).flatten() # compute all the euclidian distances between two points in X_norm
dmax = np.max(distances)
print(f'max distance between two points in feature space = {dmax}')
N_EPSILONS = 5
epsilons_range = np.linspace(dmax/100.0, dmax/20.0, num=N_EPSILONS)
# print(epsilons_range)

# minimum samples
n_samples = X_norm.shape[0]
N_MIN_SAMPLES = 10
n_min_samples = np.max( [2,n_samples/2000] )
n_max_samples = np.max( [n_samples/100, n_min_samples+1] )
min_samples_range = np.linspace(n_min_samples, n_max_samples, num=N_MIN_SAMPLES).astype('int')
# print(min_samples_range)

# silhouettes calculations
silhouette_max = -1.0

for epsilon, min_samples in product(epsilons_range, min_samples_range):
    print(f'-----------------------------------------------------')
    print(f'n_samples = {min_samples}, epsilon = {epsilon}')
    
    db = DBSCAN(
        eps=epsilon,
        min_samples=min_samples
        )
    
    labels = db.fit_predict(X_norm)
    
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # number of found clusters (without -1 which is the label for noise
    n_noise_ = list(labels).count(-1)  # number of noisy points
    noise_ratio = n_noise_/n_samples
    
    print(f'number of clusters found is {n_clusters_}')
    print(f'number of noise points found is {n_noise_} out of {n_samples} total (ie {noise_ratio*100:.2f}%)')
    
    if len(set(labels)) >= 2:
        s = silhouette_score(X_norm, labels)
        print(f'Silhouette score for {n_clusters_} clusters = {s:.3f}')
    else:
        print(f'Not calulating silhouette score as only one label found')
    
    if s > silhouette_max:
        silhouette_max = s
        opt_n_clusters = n_clusters_
        opt_eps = epsilon
        opt_min_samples = min_samples
        opt_noise_ratio = noise_ratio
        
print(f'====================================================================================================')
print(f'Best silhouette score is {silhouette_max} with {opt_n_clusters} clusters and {opt_noise_ratio*100:.2f}% noise, for epsilon = {opt_eps} and min_samples = {opt_min_samples}')

In [ ]:
print(f'Running DBSCAN with epsilon = {opt_eps} and min_samples = {opt_min_samples}.\nSilhouette score is {silhouette_max} with {opt_n_clusters} clusters and {opt_noise_ratio*100:.2f}% noise')

db = DBSCAN(
    eps=opt_eps,
    min_samples=opt_min_samples
)

labels = db.fit_predict(X_norm)

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # number of found clusters (without -1 which is the label for noise)
n_noise_ = list(labels).count(-1)  # number of noisy points
n_points = X_norm.shape[0]

print(f'=> number of clusters found is {n_clusters_}')
print(f'=> number of noise points found is {n_noise_} out of {n_points} total (ie {n_noise_/n_points*100:.2f}%)')

In [ ]:
labels = db.fit_predict(X_norm)

fig, ax = visu3d(
    X_norm=X_norm,
    model=db,
    labels=db.fit_predict(X_norm),
    titre=f'DBSCAN'
)

### Gaussian Mixtures

In [ ]:
N_MAX_GAUSSIANS = 100
ng_bics = []
bics = []

for i in range(1, N_MAX_GAUSSIANS, 5):
    n_gaussians = i+1
    gm = GaussianMixture(
        n_components=n_gaussians,
        covariance_type='full',
        random_state=42
        )
    
    labels = gm.fit_predict(X_norm)
    bic = gm.bic(X_norm)
    print(f'Model with {n_gaussians} gaussian(s) : BIC score = {bic:.0f}')
    
    ng_bics.append(n_gaussians)
    bics.append(bic)
    
opt_n_gaussians = ng_bics[np.argmin(bics)]
print(f'Best BIC score with {opt_n_gaussians}')

# BIC : https://scikit-learn.org/stable/modules/linear_model.html#aic-bic

In [ ]:
opt_n_gaussians = ng_bics[np.argmin(bics)]
print(f'Best BIC score {np.min(bics):.0f} with {opt_n_gaussians} gaussians')

fig, ax = plt.subplots()

ax.plot(ng_bics, bics)
ax.grid(True)
ax.set_title(f'BIC score v number of Gaussians in GMM model')
plt.show()

In [ ]:
gm = GaussianMixture(
    n_components=opt_n_gaussians,
    covariance_type='full',
    random_state=42
)

labels = gm.fit_predict(X_norm)

print(f'BIC score = {gm.bic(X_norm):.2f}')

In [ ]:
fig, ax = visu3d(
    X_norm=X_norm,
    model=gm,
    labels=labels,
    titre=f'GMM'
)

### UMAP : Manifold Learning => ~ visualization

In [ ]:
# DOC : https://umap.scikit-tda.org/index.html

def visu_umap(
    n_neighbors=10,
    min_dist=0.1,
    # X_norm=X_norm
):
    """Utility function to use UMAP for 3D visualization

    Args:
        n_neighbors (int, optional): _description_. Defaults to 10.
        n_components (int, optional): _description_. Defaults to 3.
        min_dist (float, optional): _description_. Defaults to 0.1.
        random_state (int, optional): _description_. Defaults to 42.
    """
    
    warnings.filterwarnings("ignore")
    
    umap_instance = umap.UMAP(
        n_neighbors=n_neighbors,
        n_components=3,
        min_dist=min_dist,
        random_state=42
        )
    
    X_new = umap_instance.fit_transform(X_norm)
    
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes(projection='3d')

    xs = X_new[:,0]
    ys = X_new[:,1]
    zs = X_new[:,2]
            
    ax.scatter3D(xs, ys, zs) #, color=c)
    titre = f'UMAP - {n_neighbors} neigbhors, {min_dist} min dist'
    ax.set_title(titre)
            
    plt.show()
    
    # return fig, ax

In [ ]:
warnings.filterwarnings("ignore")
visu_umap()

In [ ]:
ipywidgets.interact(
    visu_umap,
    n_neighbors = (5,100,10),
    min_dist = (0.1,1.0,0.05)
)